In [1]:
import os
import cv2
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import streamlit as st
from PIL import Image
import io

In [3]:

# ---------- Load Data ----------
def load_data(data_dir="data", img_size=(64, 64)):
    X, y, classes = [], [], []
    if not os.path.exists(data_dir):
        return np.array([]), np.array([]), []

    classes = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    for label, class_name in enumerate(classes):
        for img_name in os.listdir(os.path.join(data_dir, class_name)):
            img_path = os.path.join(data_dir, class_name, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None: 
                continue
            img = cv2.resize(img, img_size).astype("float32") / 255.0
            X.append(img.flatten()); y.append(label)
    return np.array(X), np.array(y), classes


In [4]:

# ---------- Train & Save Model ----------
X, y, class_names = load_data()
if len(X) > 0:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    model = SVC(kernel="linear", probability=True).fit(X_train, y_train)

    with open("image_model.pkl", "wb") as f:
        pickle.dump((model, class_names), f)

    accuracy = model.score(X_test, y_test)
    with open("accuracy.pkl", "wb") as f:
        pickle.dump(accuracy, f)

    print(f"✅ Model trained and saved! Accuracy: {accuracy:.2f}")
else:
    print("⚠️ No data found for training.")


✅ Model trained and saved! Accuracy: 0.35
